### Price an American Put option under the Heston stochastic volatility model using the Quantlib finite-differece method FdHestonVanillaEngine

In [23]:
import QuantLib as ql

# Model parameters - market and the option setup
S0 = 100.0        # Initial stock price
K = 110.0         # Strike prince
r = 0.06          # Risk-free rate
T = 1.0           # Maturity (in years)

v0 = 0.2**2       # Initial variance
kappa = 2.0       # Mean reversion speed
theta = 0.2**2    # Long-term variance
sigma = 0.1       # Vol of vol
rho = -0.7        # Correlation

# QuantLib setup
calendar = ql.NullCalendar()
day_count = ql.Actual365Fixed()
settlement_date = ql.Date.todaysDate()

# Yield/dividend/spot curves
spot_handle = ql.QuoteHandle(ql.SimpleQuote(S0))
rf_curve = ql.YieldTermStructureHandle(
    ql.FlatForward(settlement_date, r, day_count)
)
div_curve = ql.YieldTermStructureHandle(
    ql.FlatForward(settlement_date, 0.0, day_count)
)

# Heston process & model
heston_process = ql.HestonProcess(
    rf_curve, div_curve, spot_handle,
    v0, kappa, theta, sigma, rho
)
heston_model = ql.HestonModel(heston_process)

# American option setup
maturity_date = settlement_date + int(T*365)
payoff = ql.PlainVanillaPayoff(ql.Option.Put, K)
exercise = ql.AmericanExercise(settlement_date, maturity_date)

option = ql.VanillaOption(payoff, exercise)

# Create the FD pricing engine
# Grid parameters (can be adjusted for accuracy vs speed trade-off)
tGrid = 100  # Time steps
xGrid = 100  # Spot price steps
vGrid = 100   # Variance steps
dampingSteps = 0  # Damping steps for handling discontinuities

# Finite difference scheme (default: Hundsdorfer)
fd_scheme = ql.FdmSchemeDesc.Hundsdorfer()

# Create the engine
engine = ql.FdHestonVanillaEngine(
    heston_model,
    tGrid,
    xGrid,
    vGrid,
    dampingSteps,
    fd_scheme
)
option.setPricingEngine(engine)

# Calculate the price
price = option.NPV()
print(f"The American put price under Heston model: {price:.4f}")


The American put price under Heston model: 11.5123
